# <span style="color:purple">ArcGIS API for Python</span>

<img src="img/Users_02_Admin.PNG" style="width:75%"></img>

## Demo: Administrative Management of Users, Roles & Groups

### Are you frustrated with all the manual administration of users, groups, and items you have to do on your web GIS?

<img src="img/tangled_cables.jpg" style="width:100%"></img>

### Fear no more! 

### In this demo, we will go over a few workflows to show how the ArcGIS API can help administrators of Web GIS.

Let's import a few modules. We are importing getpass to provide the GIS object with a password in a safe manner.

In [29]:
from arcgis.gis import *
from getpass import getpass
from IPython.display import display

Let's call our GIS object and pass a local Portal.

In [30]:
password = getpass("Password: ")
gis = GIS("https://idt.esri.com/portal", "anieto", password, verify_cert=False)

Password: ········


### Querying for users

Let's run a full query for listed users:

In [ ]:
gis.users.search()

Filter your user search...

In [32]:
gis.users.search("nieto")

[<User username:anieto>]

You can also use wildcards in the search:

In [39]:
gis.users.search("*g")

[<User username:anieto>,
 <User username:bmajor>,
 <User username:esri_boundaries>,
 <User username:esri_demographics>,
 <User username:esri_livingatlas>,
 <User username:esri_nav>,
 <User username:portaladmin>,
 <User username:rsingh>]

Let's search for my colleague, Craig Cleveland

In [40]:
gis.users.search("Craig")

[]

### Create a new user

Users can be created through the API

In [42]:
craig = gis.users.create(username = "ccleveland", 
                           password = "GoGators4Ever",
                           firstname = "Craig",
                           lastname = "Cleveland",
                           email = "ccleveland@esri.com",
                           role = "org_user")
craig

<User username:ccleveland>

This variable is now a "user" object

In [43]:
type(craig)

arcgis.gis.User

Use intellisense to see all the options you have available with user objects.

In [ ]:
craig.

In [44]:
craig.delete()

True

### Querying for groups

In [45]:
gis.groups.search()

[<Group title:"Aviation" owner:anieto>,
 <Group title:"Esri Boundary Layers" owner:esri_boundaries>,
 <Group title:"Esri Demographic Layers" owner:esri_demographics>,
 <Group title:"Featured Maps and Apps" owner:portaladmin>,
 <Group title:"Living Atlas" owner:esri_livingatlas>,
 <Group title:"Living Atlas Analysis Layers" owner:esri_livingatlas>,
 <Group title:"Mali Data" owner:bmajor>,
 <Group title:"Navigator Maps" owner:esri_nav>,
 <Group title:"Transportation Analysis" owner:anieto>,
 <Group title:"TVA Projects" owner:bmajor>]

Let's create the group

In [46]:
# create groups
demo_group = gis.groups.create("FedGIS 2018 Demo", "Demo; National Security")

This is a "group" object

In [ ]:
demo_group.

In [47]:
type(demo_group)

arcgis.gis.Group

Like the user object, it has methods we can call on it

In [ ]:
demo_group.

## So we know how to query, create, and delete users... now what?

# Let's say your boss suddenly gets an idea!

<img src="img/idea.gif">

## Boss:
## "I want you to create a group for every team in the floor and I want to add every staff member in the DC office to it..."
## "I also want to add this to multiple portals and ArcGIS Online organizations that the office is responsible for!"
## "Also, I want to do this fast enough so that we can have it ready in two days for FedGIS."

<img src="img/tangled_cables.jpg" style="width:100%"></img>

## Have no fear! The ArcGIS API for Python is here to help.

# Step 1: Use a file containing your staff information to batch create users. This can be repeated for however many portals/organizations you need.

In [48]:
# Solution Engineer File
demo_info_excel = "data/NS_UserList.xlsx"

We're going to import the pandas library to quickly get the data from the excel

In [49]:
import pandas as pd

In [50]:
demo_dataframe = pd.read_excel(demo_info_excel)
demo_dataframe

,username,firstname,lastname,email,role
0,mwalther,Milq,Walther-Rodriguez,mwalther@esri.com,org_user
1,klevy,Kate,Levy,klevy@esri.com,org_user
2,rbernstein,Renee,Bernstein,rbernstein@esri.com,org_user
3,jfry,John,Fry,jfry@esri.com,org_user
4,tquink,Tyson,Quink,tquink@esri.com,org_user
5,agator,Albert,Gator,agator@esri.com,org_user
6,emapper,Esri,Mapper,emapper@esri.com,org_user


In [51]:
# Create an empty list that will hold our usernames for all Solution Engineers
demo_users_list = []
demo_userobjects_list = []
dummy_password = "GoGators4Ever"

# Iterate on each record of the team info file, and create a user for it
for index, row in demo_dataframe.iterrows():
    # Read from the record
    username = row['username']
    firstname = row['firstname']
    lastname = row['lastname']
    email = row['email']
    role = row['role']
    
    print("Creating user {0}...".format(username))
    demo_users_list.append(username)
    
    demo_user = gis.users.create(username=username,
                                 password=dummy_password,
                                 firstname=firstname,
                                 lastname=lastname,
                                 email=email,
                                 role=role)
    
    demo_userobjects_list.append(demo_user)

Creating user mwalther...
Creating user klevy...
Creating user rbernstein...
Creating user jfry...
Creating user tquink...
Creating user agator...
Creating user emapper...


In [52]:
demo_users_list

['mwalther', 'klevy', 'rbernstein', 'jfry', 'tquink', 'agator', 'emapper']

In [53]:
demo_userobjects_list

[<User username:mwalther>,
 <User username:klevy>,
 <User username:rbernstein>,
 <User username:jfry>,
 <User username:tquink>,
 <User username:agator>,
 <User username:emapper>]

In [54]:
for usr_obj in demo_userobjects_list:
    display(usr_obj)

<User username:mwalther>

<User username:klevy>

<User username:rbernstein>

<User username:jfry>

<User username:tquink>

<User username:agator>

<User username:emapper>

Now we use the list of solution engineer usernames to add it to the Solution Engineers group using its object's method

In [ ]:
demo_group.add_users(demo_users_list)

In [ ]:
demo_group.get_members()

## So we automatically created users, gave them privileges, created a group, and added users to the group... Now what?

# Boss: 

## "We found out that we have to remove every user from every Portal and ArcGIS Online organization we added them to.

## We also have to do it fast and leave everything nice and tidy!"

## CHOP CHOP!"

<img src="img/tenor.gif">

<img src="img/tangled_cables.jpg" style="width:100%"></img>

## Have no fear! The ArcGIS API for Python is here to help.

We're going to use the list of user objects to call their delete method and clean up after ourselves. 

Hint: This is also the way I clean up after my own self for this demo!

In [55]:
# Remove users from the se_group
demo_group.remove_users(demo_users_list)

{'notRemoved': ['mwalther',
  'klevy',
  'rbernstein',
  'jfry',
  'tquink',
  'agator',
  'emapper']}

In [56]:
# Clean up users
for user_obj in demo_userobjects_list:
    print("Removing user {0}...".format(user_obj.username))
    user_obj.delete()

Removing user mwalther...
Removing user klevy...
Removing user rbernstein...
Removing user jfry...
Removing user tquink...
Removing user agator...
Removing user emapper...


In [57]:
# Clean up group as well
demo_group.delete()

True

## You could also quickly create a script that takes a threshold for the last login date for each user, and removes the user if they have not signed in recently enough.

### ... or if you want to be nice, you can use the smtplib library in Python to send them a warning e-mail.